# Web APIs and Web Scraping

El web scraping es una técnica que podría ayudarnos a transformar datos HTML no estructurados en datos estructurados en una hoja de cálculo o base de datos.

Para algunos sitios web grandes como Airbnb, Twitter o Spotify, podemos encontrar APIs para que los desarrolladores accedan a sus datos. API (*application programming interface*) significa interfaz de programación de aplicaciones, que es el acceso para que dos aplicaciones se comuniquen entre sí. Para la mayoría de las personas, API es el enfoque más óptimo para obtener datos proporcionados por el propio sitio web.

Sin embargo, la mayoría de los sitios web no tienen servicios API. A veces, incluso si proporcionan API, los datos que podría obtener no son los que desea. Por lo tanto, escribir una secuencia de comandos de Python para crear un rastreador web se convierte en otra solución poderosa y flexible, a estas técnicas se le denomina web scraping.

# 1. Web API's

Una API, o *application programming interface*, es un forma que tienen diferentes programas de comunicarse entre ellos. Web APIs es por lo tanto, el método por que dos programas se comunican a través de internet.

Las herramientas que generlmente vamos a utilizar con las API's son: `post` y `get`. Estas herramientas permiten obtener o remitir información a una URL definida. Estas dos herramientas forman parte del módulo `requests`, que es una librería para trabajar con el protocolo HTTP (*Hypertext Transfer Protocol*).

- `get`: el método `get` solicita una representación del recurso especificado. Las solicitudes que usan `get` solo deben recuperar datos y no deben tener ningún otro efecto.

- `post`: envía datos para que sean procesados por el recurso identificado en la URI de la línea petición. Los datos se incluirán en el cuerpo de la petición. A nivel semántico está orientado a crear un nuevo recurso.

In [2]:
import requests

In [10]:
req = requests.get('https://elpais.com')
type(req)

requests.models.Response

Toda la información sobre nuestra petición está ahora almacenada en un objeto Response llamado `req`. Por ejemplo, puedes obtener la codificación de la página web usando la propiedad `.encoding`. También puedes obtener el código de estado de la petición usando la propiedad `.status_code`.

In [9]:
req.encoding, req.status_code

('utf-8', 200)

Cuando ejecutamos un método de petición, lo que se obtiene es un código de respuesta `.status_code`, que es un número que indica que ha ocurrido con la petición:

- Códigos con formato 1xx: Respuestas informativas. Indica que la petición ha sido recibida y se está procesando.
- Códigos con formato 2xx: Respuestas correctas. Indica que la petición ha sido procesada correctamente.
- Códigos con formato 3xx: Respuestas de redirección. Indica que el cliente necesita realizar más acciones para finalizar la petición.
- Códigos con formato 4xx: Errores causados por el cliente. Indica que ha habido un error en el procesado de la petición a causa de que el cliente ha hecho algo mal.
- Códigos con formato 5xx: Errores causados por el servidor. Indica que ha habido un error en el procesado de la petición a causa de un fallo en el servidor.

También se puede acceder al contenido de la petición mediante el atributo `.content`, o `.text`.

In [15]:
req.content[:500]

b'<!DOCTYPE html><html lang="es"><head><title>EL PA\xc3\x8dS: el peri\xc3\xb3dico global</title><meta name="lang" content="es"/><meta name="author" content="Ediciones El Pa\xc3\xads"/><meta name="robots" content="index,follow"/><meta name="description" content="Noticias de \xc3\xbaltima hora sobre la actualidad en Espa\xc3\xb1a y el mundo: pol\xc3\xadtica, econom\xc3\xada, deportes, cultura, sociedad, tecnolog\xc3\xada, gente, opini\xc3\xb3n, viajes, moda, televisi\xc3\xb3n, los blogs y las firmas de EL PA\xc3\x8dS. Adem\xc3\xa1s especiales, v\xc3\xaddeos, fotos, audios, gr'

## 1.1. Acceder a una API

Para explicar parte se va a realizar mediante un ejercicio de tipo práctico. En la url http://open-notify.org/ podemos encontrar tres APIs que arrojan información relativa a la estación espacial internacional (*ISS*).

En el primer ejercicio vamos a trabajar con l API *International Space Station Current Location*, la cual da la coordenadas a cada momento de la posición de la ISS. Para obtener esta información en cualquier programa de python, lo único que tendremos que realizar es un `get()`.

In [23]:
url = 'http://api.open-notify.org/iss-now.json'
iss_location = requests.get(url)
iss_location.content

b'{"iss_position": {"longitude": "-80.8566", "latitude": "32.6177"}, "timestamp": 1603731027, "message": "success"}'

El contenido de la petición es string con formato json, por lo que podemos acceder a él mediante la librería `json` y trabajar con sus elementos.

In [24]:
import json

iss_location_json = json.loads(iss_location.content)
iss_location_json

{'iss_position': {'longitude': '-80.8566', 'latitude': '32.6177'},
 'timestamp': 1603731027,
 'message': 'success'}

In [25]:
iss_location_json['iss_position']['latitude']

'32.6177'

También es accesible esta información mediante el método `.json()` de la request.

In [36]:
iss_location.json()

{'iss_position': {'longitude': '-80.8566', 'latitude': '32.6177'},
 'timestamp': 1603731027,
 'message': 'success'}

**Ejercicio:**

Escribe una funión que devuelva la duration de los 5 próximos pases de la ISS para una latitud y longitud dada. Use http://open-notify.org/Open-Notify-API/ISS-Pass-Times/

Necesitamos parametrizar las coordenadas en la petición como se define en las especificaciones de la API. Por ejemplo para Madrid:

http://api.open-notify.org/iss-pass.json?lat=40.4&lon=-3.7

In [29]:
# http://api.open-notify.org/iss-pass.json?lat=LAT&lon=LON
# http://api.open-notify.org/iss-pass.json?lat=40.4&lon=-3.7

def iss_pass_times(lat, lon):
    url = f'http://api.open-notify.org/iss-pass.json?lat={lat}&lon={lon}'
    iss_pass = requests.get(url)
    iss_pass_json = json.loads(iss_pass.content)
    passes = iss_pass_json['response']
    return passes

iss_pass_times('40.4', '-3.7')

[{'duration': 598, 'risetime': 1603768345},
 {'duration': 646, 'risetime': 1603774117},
 {'duration': 574, 'risetime': 1603779999},
 {'duration': 563, 'risetime': 1603785877},
 {'duration': 637, 'risetime': 1603791694}]

Aunque en el caso anterior resulta sencillo codificar los parámetros dentro de la URL, a veces puede ser una tarea más complicada y propiciar errores en la codificación. Para solventar este problema, el módulo `requests` permite pasar los parámetros mediante un diccionario como argumentos de la función `get()`.

In [30]:
madrid = {'lat': 40, 'lon': -3}

response = requests.get('http://api.open-notify.org/iss-pass.json', params = madrid)
response.content

b'{\n  "message": "success", \n  "request": {\n    "altitude": 100, \n    "datetime": 1603731644, \n    "latitude": 40.0, \n    "longitude": -3.0, \n    "passes": 5\n  }, \n  "response": [\n    {\n      "duration": 607, \n      "risetime": 1603768342\n    }, \n    {\n      "duration": 642, \n      "risetime": 1603774123\n    }, \n    {\n      "duration": 564, \n      "risetime": 1603780011\n    }, \n    {\n      "duration": 557, \n      "risetime": 1603785890\n    }, \n    {\n      "duration": 636, \n      "risetime": 1603791705\n    }\n  ]\n}\n'

En otros casos, la parametrización de la request puede ser tan complicada, que sea necesario enviarlos en formato json o con un formulario HTML, mediante la herramienta `post` mencionada al principio del notebook se puede realicar esta solicitud.

In [38]:
payload = {'key1': 'value1', 'key2': 'value2'}
r = requests.post("http://httpbin.org/post", data=payload)
r.text

'{\n  "args": {}, \n  "data": "", \n  "files": {}, \n  "form": {\n    "key1": "value1", \n    "key2": "value2"\n  }, \n  "headers": {\n    "Accept": "*/*", \n    "Accept-Encoding": "gzip, deflate", \n    "Content-Length": "23", \n    "Content-Type": "application/x-www-form-urlencoded", \n    "Host": "httpbin.org", \n    "User-Agent": "python-requests/2.24.0", \n    "X-Amzn-Trace-Id": "Root=1-5f970370-41c0974f78e117535e40efb9"\n  }, \n  "json": null, \n  "origin": "83.50.213.41", \n  "url": "http://httpbin.org/post"\n}\n'

# 2. Web Scraping

En un navegador lo que hacemos es escribir una URL, lo que hace que se envíe una petición siguiendo el protocolo HTTP, a un servidor, el cual nos devuelve el código HTML que nuestro navegador consigue interpretar y transformar con ese aspecto visual que vemos en las páginas web.

Con Python podemos hacer exactamente lo mismo, crear programas que generen peticiones al servidor y recibir el código fuente en formato HTML.

En general, un código HTML, el código fuente de una página, contiene muchísima información, de la cual solo nos interesan ciertas partes. Es en este código dónde mediante diferentes librerías, podremos "arañar" la información que buscamos según una base de parámetros.

Una de las librerías más utilizas para llevar a cabo web scraping es `BeautifulSoup`.

In [50]:
from bs4 import BeautifulSoup

Para poder encontrar dentro del código HTML la información del elemento que nos interesa, es necesario inspeccionarlo (botón derecho del ratón) y encontrar su TAG *(<>, </>)* y jerarquía de etiquetas dentro del código.

<img src="_images\web_scraping_inspect.png" alt="Drawing" style="width: 800px;"/>

## 2.1. Ética del Web Scraping

Ahora que tenemos los módulos que necesitamos, podemos empezar a realizar web scraping, pero primero, una serie de conductas éticas:

- Es necesario revisar los términos y condiciones del sitio web que vayamos a scrapear. Son sus datos, y probablemente tengan algun reglamento de gobiernos del datos sobre ellos.

- Se prudente, un ordenador es capaz de solicitar requests mucho más rápido que cualquier humano. Asegurate de espaciar tus requests en el tiempo para no sobrecargar los servidores.

- La estructura de las páginas web cambia constantemente, por lo que debes estar preparado para reescribir tu código. Adicionalmente, las webs suelen presentar inconsistencias, por lo que muchas veces es necesario limpiar los datos un vez se han obtenido.

## 2.2. Métodos BeautifulSoup 

Dentro de `BeautifulSoup` podemos encontrar dos métodos para buscar información:

- `find()`: encuentra el primer TAG coincidente, y devuelve un TAG object
- `find_all()`: encuentra todos los TAG coincidentes, y devuelve un ResultSet object

Una vez se tienen los TAG, se puede extraer información de los mismos mediante los siguientes métodos:

- `.text`: extrae el texto del TAG y devuelve un string
- `.content`: extrae todos los hijos del TAG, y devuelve una lista de TAGs y strings

In [43]:
req = requests.get('https://www.elmundotoday.com/')
soup = BeautifulSoup(req.content)

In [45]:
section_header = soup.find('ul')
section_header

<ul class="td-mobile-main-menu" id="menu-menu-mobile"><li class="menu-item menu-item-type-custom menu-item-object-custom menu-item-first menu-item-65940" id="menu-item-65940"><a href="https://www.elmundotoday.com/login/">Iniciar sesión</a></li>
<li class="menu-item menu-item-type-taxonomy menu-item-object-category menu-item-65931" id="menu-item-65931"><a href="https://www.elmundotoday.com/noticias/internacional/">Internacional</a></li>
<li class="menu-item menu-item-type-taxonomy menu-item-object-category menu-item-65932" id="menu-item-65932"><a href="https://www.elmundotoday.com/noticias/espanya/">España</a></li>
<li class="menu-item menu-item-type-taxonomy menu-item-object-category menu-item-65933" id="menu-item-65933"><a href="https://www.elmundotoday.com/noticias/sociedad/">Sociedad</a></li>
<li class="menu-item menu-item-type-taxonomy menu-item-object-category menu-item-65934" id="menu-item-65934"><a href="https://www.elmundotoday.com/noticias/tecnologia/">Ciencia y Tecnología</a>

In [46]:
for section in section_header.find_all('li'):
    print(section.text)
    print(section.find('a')['href'])

Iniciar sesión
https://www.elmundotoday.com/login/
Internacional
https://www.elmundotoday.com/noticias/internacional/
España
https://www.elmundotoday.com/noticias/espanya/
Sociedad
https://www.elmundotoday.com/noticias/sociedad/
Ciencia y Tecnología
https://www.elmundotoday.com/noticias/tecnologia/
Cultura
https://www.elmundotoday.com/noticias/cultura/
Gente
https://www.elmundotoday.com/noticias/gente/
Deportes
https://www.elmundotoday.com/noticias/deportes/
Vídeos
https://www.elmundotoday.com/noticias/videos/


In [48]:
results = []
for headline in soup.find_all('h3'):
    text = headline.text
    url = headline.find('a')['href']
    results.append((text, url))
    
results

[('Un salón, un bar y una clase: así  contagia el fascismo',
  'https://www.elmundotoday.com/2020/10/un-salon-un-bar-y-una-clase-asi-contagia-el-fascismo/'),
 ('Los chilenos votan a favor de modificar la Constitución para que Felipe VI sea su rey',
  'https://www.elmundotoday.com/2020/10/los-chilenos-votan-a-favor-de-modificar-la-constitucion-para-que-felipe-vi-sea-su-rey/'),
 ('El Gobierno informa de que incumplir el toque de queda supondrá penalti a favor del Real Madrid',
  'https://www.elmundotoday.com/2020/10/el-gobierno-informa-de-que-incumplir-el-toque-de-queda-supondra-penalti-a-favor-del-real-madrid/'),
 ('España se da cuenta ahora de que el «botellón» representa el 80% de su PIB',
  'https://www.elmundotoday.com/2020/10/espana-se-da-cuenta-ahora-de-que-el-botellon-representa-el-80-de-su-pib/'),
 ('Desalojan el Congreso por una inundación de lágrimas de facha',
  'https://www.elmundotoday.com/2020/10/desalojan-el-congreso-por-una-inundacion-de-lagrimas-de-facha/'),
 ('El Mundo

https://github.com/emunozlorenzo/MasterDataScience/tree/master/10_Web_Scraping

# X. Bibliografía

- https://requests.readthedocs.io/es/latest/
- https://es.wikipedia.org/wiki/Protocolo_de_transferencia_de_hipertexto
- https://github.com/emunozlorenzo/MasterDataScience/tree/master/10_Web_Scraping